In [ ]:
!pip install ultralytics roboflow matplotlib seaborn --quiet

In [ ]:
import torch
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="uiIpENtp73grZiYxNBSl")
project = rf.workspace("engineer-bi8td").project("plastic-multiclass")
version = project.version(2)
dataset = version.download("yolov8")


In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Load the largest model for best accuracy
model = YOLO('yolov8x.pt')  # XLarge model (best for >90% accuracy)

# Train with high-accuracy settings
results = model.train(
    data=f"{dataset.location}/data.yaml",  # Auto path if using Roboflow
    epochs=300,                # More epochs for convergence
    imgsz=1024,                # Higher resolution for small objects
    batch=8,                   # Smaller batch for RTX 4050 (6GB VRAM)
    optimizer='AdamW',         # Better optimization
    lr0=0.01,                  # Learning rate
    lrf=0.01,                  # Final learning rate
    momentum=0.937,            # SGD momentum
    weight_decay=0.0005,       # Regularization
    warmup_epochs=3.0,         # Gradual learning start
    warmup_momentum=0.8,       # Initial momentum
    box=7.5,                   # Box loss gain
    cls=0.5,                   # Class loss gain
    dfl=1.5,                   # Distribution Focal Loss gain
    augment=True,              # Advanced augmentations
    flipud=0.5,                # Vertical flip probability
    fliplr=0.5,                # Horizontal flip probability
    mosaic=1.0,                # Mosaic augmentation probability
    mixup=0.1,                 # MixUp augmentation probability
    copy_paste=0.1,            # Copy-Paste augmentation probability
    erasing=0.4,               # Random erasing probability
    degrees=45.0,              # Rotation degrees
    translate=0.2,             # Image translation (+/- fraction)
    scale=0.2,                 # Image scale (+/- gain)
    shear=10.0,                # Image shear (+/- deg)
    perspective=0.0001,        # Image perspective (+/- fraction)
    hsv_h=0.015,               # Image HSV-Hue augmentation (fraction)
    hsv_s=0.7,                 # Image HSV-Saturation augmentation (fraction)
    hsv_v=0.4,                 # Image HSV-Value augmentation (fraction)
    device='0',                # Use GPU
    workers=8,                 # Data loading threads
    single_cls=False,          # Treat as multi-class
    rect=False,                # Rectangular training
    cos_lr=True,               # Cosine LR scheduler
    close_mosaic=10,           # Disable mosaic last epochs
    resume=False,              # Resume from last.pt
    amp=True,                  # Automatic Mixed Precision
)

In [ ]:
# Plot training results
results = model.train()  # Continue training if needed

# Show performance metrics
metrics = model.val()
print(f"mAP50-95: {metrics.box.map:.2f}")  # Should be >0.9 for 90% accuracy

In [ ]:
from IPython.display import Image

# Show confusion matrix
Image(filename=f'{model.trainer.save_dir}/confusion_matrix.png')

# Show training curves
Image(filename=f'{model.trainer.save_dir}/results.png')

 Running Inference (After Training)

In [ ]:
# Load best model
best_model = YOLO(f'{model.trainer.save_dir}/weights/best.pt')

# Run prediction
results = best_model.predict(
    source="path/to/test/image.jpg",
    conf=0.5,        # Confidence threshold
    iou=0.45,        # NMS IoU threshold
    imgsz=1024,      # Inference size
    device='0',      # Use GPU
    show=True,       # Display results
    save=True,       # Save results
    save_txt=True,   # Save results as .txt
    save_conf=True,  # Save confidence scores
    save_crop=False, # Save cropped predictions
    show_labels=True,
    show_conf=True,
    line_width=2,
)

# Display results in notebook
for result in results:
    display(Image.fromarray(result.plot()[:, :, ::-1]))

Saving Progress:

Model auto-saves every epoch in runs/detect/train/weights/

Best model is best.pt